In [ ]:
import mysql.connector
from pymongo import MongoClient

myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor(buffered=True)
mycursor.execute("SELECT DISTINCT  d.id_juego, d.nombre, d.descripcion, d.clasificacion, d.precio FROM juegos d JOIN claves_juegos c ON c.id_juego = d.id_juego WHERE c.vendido = 1")
vista = mycursor.fetchall()

In [ ]:
def valoraciones(num):
    valoracion = mycol.find({"biblioteca.id_juego":int(num)},{"biblioteca.$.valoraciones":1})
    max = -1
    min = 9999
    commax = ""
    commin = ""
    todas = []
    maxval = ""
    minval = ""
    if(valoracion.count() > 0):
        for x in valoracion:
            todas.append("Valoración: "+ str(x['biblioteca'][0]['valoraciones']) + " - '"+ x['biblioteca'][0]['comentarios'] + "'")
            if(x['biblioteca'][0]['valoraciones'] > max):
                max = x['biblioteca'][0]['valoraciones']
                commax = x['biblioteca'][0]['comentarios']
            if(x['biblioteca'][0]['valoraciones'] < min):
                min = x['biblioteca'][0]['valoraciones']
                commin = x['biblioteca'][0]['comentarios']
        maxval = "Máxima valoración: " + str(max) + " - '" + commax + "'"
        minval = "Mínima valoración: " + str(min) + " - '" + commin + "'"
    return (todas,maxval, minval, valoracion.count())

In [ ]:
from bottle import Bottle, route, run, template, request ,post ,get
import os

@route('/')
def Juegos():
    return template(os.getcwd()+'\\VistaJuegos.tpl', vista=vista)

@post('/Valoraciones')
def Valoraciones():
    id_juego = request.forms.get('id_juego')
    result = valoraciones(id_juego)
    return template(os.getcwd()+'\\VistaValoraciones.tpl', todas = result[0], maxval = result[1], minval = result[2], contador = result[3])
run(host='localhost', port=8080)